# Modo analítico 

# Normalização dos atributos

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xlrd
from sklearn import datasets, linear_model
workbook = xlrd.open_workbook('/home/mmrosa/MEGA/MEGAsync/GitHub Commits/TEIA/Teia 18-02-2017/Concrete_Data.xls')
sheet = workbook.sheet_by_index(0)

def normalGaussiana(X):
    
    dp = np.std(X)
    a = 1/(dp*np.sqrt(2*np.pi))
    media = np.mean(X)
    (n,m) = np.shape(X)  
    
    for i in range(n):
        for j in range(m):
            X[i][j] = a*(np.exp(1)**(1/2))**(((X[i][j] - media)/dp)**2)
    
    return X

# Algoritmo

In [3]:
class LMSTrainer:
     
    def _init_(self):
        pass
        
    def h(self,thetas,linhaMatriz):
    
        hip = 0
        hip = hip + np.dot(thetas, [1]+linhaMatriz)
        
        return hip
    
    def somatorio(self, thetas, exemplos, gabarito, m):
        
        resultado = 0
        
        for i in range(m):
            resultado = resultado + (self.h(thetas, exemplos[i]) - gabarito[i]) ** 2
        return resultado
    
    def somat(self, thetas, exemplos, gabarito, j):
        
        resultado = 0
        linha = len(gabarito)
        
        for i in range(linha):
            if (j-1) < 0: # se j for 0, significa que estou calculando Theta(0), 0 - 1 < 0
                resultado = resultado + self.h(thetas, exemplos[i]) - gabarito[i]
            else:
                # faço j-1 porque não coloquei uma coluna de 1 para multiplicar o Theta(0) na matriz original, logo
                # meu j vai ser maior que o num de colunas na matriz original e acessaria índice indevido
                resultado = resultado + (self.h(thetas, exemplos[i]) - gabarito[i])*exemplos[i][j-1]
                #print("exemplo[i][j-1] =", exemplos[i][j-1])
                
        return resultado
    
    def custo(self, thetas, exemplos, gabarito, qtdElementos):
        
        m = qtdElementos
        valor = (1/(2*m)) * self.somatorio(thetas, exemplos, gabarito, m)
        return valor
        
    def train (self, thetas, exemplos, gabarito):
        
        linhas = len(gabarito)
        alfa = 0.000001
        aux = [0] * len(thetas)
        m = linhas
        inicial = self.custo(thetas, exemplos, gabarito, m)
        final = 0
        #print("Custo inicial: ",inicial)
        interacoes = 0
        # final > 0.000001 or 
        while(interacoes < 2000):
            for i in range(len(thetas)):
                coluna = i
                aux[i] = thetas[i] - (alfa/m) * self.somat(thetas, exemplos, gabarito, coluna)
            thetas = aux[:]
            j = self.custo(thetas, exemplos, gabarito,m)
            interacoes = interacoes + 1
            final = self.custo(thetas, exemplos, gabarito, m)
            
        #print("Custo final:", final)
        return LMSPredictor(thetas)
    
    def analitic(self,exemplos,gabarito):
        
        for i in range(len(exemplos)):
            exemplos[i] = [1]+exemplos[i]
            
        matrizTransposta = np.transpose(exemplos) 
        produtoMatrizes = np.dot(matrizTransposta,exemplos) 
        matrizInversa = np.linalg.inv(produtoMatrizes)
       
        aux = np.dot(matrizInversa,matrizTransposta) 
        vetorThetas = np.dot(aux,gabarito)
       
        return LMSPredictor(vetorThetas)
    
class LMSPredictor:
        
    def __init__(self, thetas):
        self.thetas = thetas
    
    def predict (self, samples):
        valores = [0] * len(samples)
        for i in range(len(samples)):
            valores[i] = np.dot(self.thetas, [1]+samples[i])
            valores[i] = np.round(valores[i],4)
        return valores


In [6]:
matriz = []
y = []

# Pega a matriz, exceto a última coluna
for i in range(sheet.nrows-1):
    matriz.append([])
    for j in range(sheet.ncols):
        #print(sheet.cell_value(i+1,j))
        matriz[i].append(sheet.cell_value(i+1,j))
        
#Normaliza os atributos
matrix = []
matrix = normalGaussiana(matriz)

# Pega o gabarito
for i in range(1030):
    y.append(matrix[i][8])
    
#Retira uma coluna da matriz
for i in range(1030):
    matrix[i] = matrix[i][0:8]
    
thetas = [0,1,1,1,1,1,1,1,1]
conjunto_treino = matrix[:500]
gabarito_treino = y[:500]
conjunto_teste  = matrix[301:310]
gabarito_teste  = y[301:310]

for i in range(len(gabarito_teste)):
    gabarito_teste[i] = np.round(gabarito_teste[i],4)

lms = LMSTrainer()
preditor_analitico = lms.analitic(conjunto_treino, gabarito_treino)


print("Gabarito:", gabarito_teste)
print()
print("Resposta Analítico:", preditor_analitico.predict(conjunto_teste))

# Plot 
#plt.scatter(conjunto_teste, gabarito_teste,  color='black')
#plt.plot(conjunto_teste ,regression.predict(conjunto_teste), color='blue',linewidth=3)

#plt.xticks(())
#plt.yticks(())
#plt.show()
    

Gabarito: [0.0015, 0.0014, 0.0014, 0.0015, 0.0015, 0.0014, 0.0014, 0.0014, 0.0015]

Resposta Analítico: [0.0015, 0.0015, 0.0014, 0.0015, 0.0015, 0.0015, 0.0015, 0.0014, 0.0015]


# Métricas

In [10]:
import numpy as np
from numpy import sqrt

def mae(resposta, predicao):
    n = len(resposta)
    soma = .0
    dif = 0
    for i in range(n):
        dif = resposta[i] - predicao[i]
        soma = soma + abs(dif)
    return soma/n

def mse(resposta, predicao):
    soma = .0
    dif = 0
    n = len(resposta)
    for i in range(n):
        dif = (resposta[i] - predicao[i])**2
        soma = soma+dif
    return soma/n

def rmse(resposta, predicao):
    return sqrt(mse(resposta, predicao))

def mape(resposta, predicao):
    soma = .0
    dif = 0
    n = len(resposta)
    
    for i in range(n):
        dif = abs((resposta[i] - predicao[i])/resposta[i])
        soma = soma + dif
    return soma/n

print(mae (gabarito_teste,preditor_analitico.predict(conjunto_teste)))
print(mse (gabarito_teste,preditor_analitico.predict(conjunto_teste)))
print(rmse(gabarito_teste,preditor_analitico.predict(conjunto_teste)))
print(mape (gabarito_teste,preditor_analitico.predict(conjunto_teste)))

3.33333333333e-05
3.33333333333e-09
5.7735026919e-05
0.0238095238095
